In [1]:
!pip install kora -q

     |████████████████████████████████| 57 kB 2.2 MB/s 
     |████████████████████████████████| 60 kB 4.8 MB/s 


In [2]:
import torch
from torch.utils.data import Dataset
import os
import pickle
import argparse
from collections import defaultdict
import pandas as pd
import numpy as np
import gzip
import shutil
import spacy 
import pandas as pd 
from tqdm import tqdm
from collections import Counter
import kora.install.rdkit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
import torch
import math
import torch.nn as nn
from rdkit import Chem
from rdkit import rdBase
rdBase.DisableLog('rdApp.*')

def split(sm):
    sm_couples = []
    i = 0
    while i < len(sm):
        if i == len(sm) - 1:
            sm_couples.append(sm[i])
            i += 1
        elif sm[i] == '%':
            sm_couples.append(sm[i:i+3])
            i += 3
        elif sm[i:i+2] in set(['Cl', 'Ca', 'Cu', 'Br', 'Be', 'Ba', 'Bi', 'Si', 'Se', 'Sr', 'Na', 'Ni', 'Rb', 'Ra', 'Xe', 'Li', 'Al', ' As', 'Ag', 'Au',\
                            'Mg', 'Mn', 'Te', 'Zn', 'si', 'se', 'te', 'He', 'Kr', 'Fe', '+2', '+3', '+4', '-2', '-3', '-4']):
            sm_couples.append(sm[i:i+2])
            i += 2
        else:
            sm_couples.append(sm[i])
            i += 1
    return ' '.join(sm_couples)

# Smile Enumerator

In [48]:
class Iterator(object):
    """Abstract base class for data iterators.
    # Arguments
        n: Integer, total number of samples in the dataset to loop over.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seeding for data shuffling.
    """

    def __init__(self, n, batch_size, shuffle, seed):
        self.n = n
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.batch_index = 0
        self.total_batches_seen = 0
        self.index_generator = self._flow_index(n, batch_size, shuffle, seed)
        if n < batch_size:
            raise ValueError('Input data length is shorter than batch_size\nAdjust batch_size')

    def reset(self):
        self.batch_index = 0

    def _flow_index(self, n, batch_size=32, shuffle=False, seed=None):
        # Ensure self.batch_index is 0.
        self.reset()
        while 1:
            if seed is not None:
                np.random.seed(seed + self.total_batches_seen)
            if self.batch_index == 0:
                index_array = np.arange(n)
                if shuffle:
                    index_array = np.random.permutation(n)

            current_index = (self.batch_index * batch_size) % n
            if n > current_index + batch_size:
                current_batch_size = batch_size
                self.batch_index += 1
            else:
                current_batch_size = n - current_index
                self.batch_index = 0
            self.total_batches_seen += 1
            yield (index_array[current_index: current_index + current_batch_size],
                   current_index, current_batch_size)

    def __iter__(self):
        # Needed if we want to do something like:
        # for x, y in data_gen.flow(...):
        return self

    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)

class SmilesIterator(Iterator):
    """Iterator yielding data from a SMILES array.
    # Arguments
        x: Numpy array of SMILES input data.
        y: Numpy array of targets data.
        smiles_data_generator: Instance of `SmilesEnumerator`
            to use for random SMILES generation.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        dtype: dtype to use for returned batch. Set to keras.backend.floatx if using Keras
    """

    def __init__(self, x, y, smiles_data_generator,
                 batch_size=32, shuffle=False, seed=None,
                 dtype=np.float32
                 ):
        if y is not None and len(x) != len(y):
            raise ValueError('X (images tensor) and y (labels) '
                             'should have the same length. '
                             'Found: X.shape = %s, y.shape = %s' %
                             (np.asarray(x).shape, np.asarray(y).shape))

        self.x = np.asarray(x)

        if y is not None:
            self.y = np.asarray(y)
        else:
            self.y = None
        self.smiles_data_generator = smiles_data_generator
        self.dtype = dtype
        super(SmilesIterator, self).__init__(x.shape[0], batch_size, shuffle, seed)

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros(tuple([current_batch_size] + [ self.smiles_data_generator.pad, self.smiles_data_generator._charlen]), dtype=self.dtype)
        for i, j in enumerate(index_array):
            smiles = self.x[j:j+1]
            x = self.smiles_data_generator.transform(smiles)
            batch_x[i] = x

        if self.y is None:
            return batch_x
        batch_y = self.y[index_array]
        return batch_x, batch_y


class SmilesEnumerator(object):
    """SMILES Enumerator, vectorizer and devectorizer
    
    #Arguments
        charset: string containing the characters for the vectorization
          can also be generated via the .fit() method
        pad: Length of the vectorization
        leftpad: Add spaces to the left of the SMILES
        isomericSmiles: Generate SMILES containing information about stereogenic centers
        enum: Enumerate the SMILES during transform
        canonical: use canonical SMILES during transform (overrides enum)
    """
    def __init__(self, charset = '@C)(=cOn1S2/H[N]\\', pad=120, leftpad=True, isomericSmiles=True, enum=True, canonical=False):
        self._charset = None
        self.charset = charset
        self.pad = pad
        self.leftpad = leftpad
        self.isomericSmiles = isomericSmiles
        self.enumerate = enum
        self.canonical = canonical

    @property
    def charset(self):
        return self._charset
        
    @charset.setter
    def charset(self, charset):
        self._charset = charset
        self._charlen = len(charset)
        self._char_to_int = dict((c,i) for i,c in enumerate(charset))
        self._int_to_char = dict((i,c) for i,c in enumerate(charset))
        
    def fit(self, smiles, extra_chars=[], extra_pad = 5):
        """Performs extraction of the charset and length of a SMILES datasets and sets self.pad and self.charset
        
        #Arguments
            smiles: Numpy array or Pandas series containing smiles as strings
            extra_chars: List of extra chars to add to the charset (e.g. "\\\\" when "/" is present)
            extra_pad: Extra padding to add before or after the SMILES vectorization
        """
        charset = set("".join(list(smiles)))
        self.charset = "".join(charset.union(set(extra_chars)))
        self.pad = max([len(smile) for smile in smiles]) + extra_pad
        
    def randomize_smiles(self, smiles):
        """Perform a randomization of a SMILES string
        must be RDKit sanitizable"""
        m = Chem.MolFromSmiles(smiles)
        if m is None:
          return None
        ans = list(range(m.GetNumAtoms()))
        np.random.shuffle(ans)
        nm = Chem.RenumberAtoms(m,ans)
        return Chem.MolToSmiles(nm, canonical=self.canonical, isomericSmiles=self.isomericSmiles)

    def transform(self, smiles):
        """Perform an enumeration (randomization) and vectorization of a Numpy array of smiles strings
        #Arguments
            smiles: Numpy array or Pandas series containing smiles as strings
        """
        one_hot =  np.zeros((smiles.shape[0], self.pad, self._charlen),dtype=np.int8)
        
        if self.leftpad:
            for i,ss in enumerate(smiles):
                if self.enumerate: ss = self.randomize_smiles(ss)
                l = len(ss)
                diff = self.pad - l
                for j,c in enumerate(ss):
                    one_hot[i,j+diff,self._char_to_int[c]] = 1
            return one_hot
        else:
            for i,ss in enumerate(smiles):
                if self.enumerate: ss = self.randomize_smiles(ss)
                for j,c in enumerate(ss):
                    one_hot[i,j,self._char_to_int[c]] = 1
            return one_hot

      
    def reverse_transform(self, vect):
        """ Performs a conversion of a vectorized SMILES to a smiles strings
        charset must be the same as used for vectorization.
        #Arguments
            vect: Numpy array of vectorized SMILES.
        """       
        smiles = []
        for v in vect:
            #mask v 
            v=v[v.sum(axis=1)==1]
            #Find one hot encoded index with argmax, translate to char and join to string
            smile = "".join(self._int_to_char[i] for i in v.argmax(axis=1))
            smiles.append(smile)
        return np.array(smiles)

# Data Loader

In [49]:
class Randomizer(object):
    def __init__(self):
        self.smile_enum = SmilesEnumerator()
    
    def __call__(self, smile):
        random_smile = self.smile_enum.randomize_smiles(smile)
        if random_smile is None:
            sm_spaced = split(smile)
        else:
            sm_spaced = split(random_smile)
        sm_split = sm_spaced.split()

        if len(sm_split) <= 218:
            return sm_split
        else:
            return split(sm).split()

    def random_transform(self, sm):
        return self.smile_enum.randomize_smiles(sm)

class MyDataset(Dataset):
    def __init__(self, smiles, vocab, seq_len=220, random=Randomizer()):
        self.smiles = smiles
        self.vocab = vocab
        self.seq_len = seq_len
        self.random = random

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, item):
        sm = self.smiles[item]
        if sm is None:
          return None
        sm = self.random(sm)
        content = [self.vocab.word_dict.get(token, self.vocab.unk_index) for token in sm]
        X = [self.vocab.sos_index] + content + [self.vocab.eos_index]
        padding = [self.vocab.pad_index] * (self.seq_len - len(X))
        X.extend(padding)
        return torch.tensor(X)

Build Vocab

In [50]:
class Vocab(object):
    def __init__(self, texts, min_freq):
        self.pad_index = 0
        self.unk_index = 1
        self.eos_index = 2
        self.sos_index = 3
        self.mask_index = 4

        counter = Counter()
        
        self.words = ["<pad>", "<unk>", "<eos>", "<sos>", "<mask>"]

        for line in texts:
            if isinstance(line, list):
                words = line
            else:
                words = line.replace("\t", "").replace("\n", "").split()

            for word in words:
                if word not in self.words:
                    counter[word] += 1

        freq_list = counter.most_common()
        
        for word, freq in freq_list:
            if freq < min_freq:
                break
            self.words.append(word)

        self.word_dict = {tok: i for i, tok in enumerate(self.words)}

    def __len__(self):
        return len(self.words)

    def extend(self, vocab_new, sort=False):
        words_new = sorted(vocab_new.words) if sort else vocab_new.words
        for w in words_new:
            if w not in self.words:
                self.words.append(w)
                self.words_dict[w] = len(self.words) - 1

    def load_vocab(vocab_path: str):
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            pickle.dump(self, f)

In [51]:
PAD = 0
UNK = 1
EOS = 2
SOS = 3
MASK = 4

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout = 0.1, max_len = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class SmTrfm(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_layers, dropout=0.1):
        super(TrfmSeq2seq, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(in_size, hidden_size)
        self.pe = PositionalEncoding(hidden_size, dropout)
        self.trfm = nn.Transformer(d_model=hidden_size, nhead=4, 
        num_encoder_layers=n_layers, num_decoder_layers=n_layers, dim_feedforward=hidden_size)
        self.out = nn.Linear(hidden_size, out_size)

    def forward(self, src):
        embedded = self.embed(src)
        embedded += self.pe(embedded)
        hidden = self.trfm(embedded, embedded)
        out = self.out(hidden)
        out = F.log_softmax(out, dim=2)
        return out

    def _encode(self, src):
        embedded = self.embed(src)
        embedded = self.pe(embedded)
        output = embedded
        for i in range(self.trfm.encoder.num_layers - 1):
            output = self.trfm.encoder.layers[i](output, None)
        penul = output.detach().numpy()
        output = self.trfm.encoder.layers[-1](output, None)
        if self.trfm.encoder.norm:
            output = self.trfm.encoder.norm(output)
        output = output.detach().numpy()
        
        return np.hstack([np.mean(output, axis=0), np.max(output, axis=0), output[0,:,:], penul[0,:,:]])
    
    def encode(self, src):
        batch_size = src.shape[1]
        if batch_size <= 100:
            return self._encode(src)
        else: # Batch is too large to load
            i = 0
            while i < batch_size:
                if o == 0:
                    out = self._encode(src[:, i:i+100])
                else:
                    out = np.concatenate([out, self._encode(src[:, i:i+100])], axis=0)
                i += 100
            return out


In [52]:
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F

class SandwichTransformer(nn.Module):

    def __init__(self, d_model = 512, nhead = 8, num_encoder_layers = 6,
                 num_decoder_layers = 6, dim_feedforward = 2048, dropout = 0.1,
                 sandwich_k = 2, sandwich_encoder = False, sandwich_decoder = False,
                 activation = F.relu, layer_norm_eps = 1e-5):
        super(SandwichTransformer, self).__init__()

        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                activation, layer_norm_eps)
        encoder_norm = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm, sandwich_k if sandwich_encoder else 0)

        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                activation, layer_norm_eps)
        decoder_norm = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm, sandwich_k if sandwich_decoder else 0)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

        self.sandwich_k = sandwich_k
        self.sandwich_encoder = sandwich_encoder
        self.sandwich_decoder = sandwich_decoder

    def forward(self, src, tgt, src_mask = None, tgt_mask = None, memory_mask = None,
                src_key_padding_mask = None, tgt_key_padding_mask = None,
                memory_key_padding_mask = None):

        is_batched = src.dim() == 3

        memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask, )
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
        return output


    def generate_square_subsequent_mask(sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

class TransformerEncoder(nn.Module):
    _constants_ = ['norm']
    def __init__(self, encoder_layer, num_layers, norm=None, sandwich_k=0):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(encoder_layer, num_layers+sandwich_k)
        self.num_layers = num_layers+sandwich_k
        self.norm = norm
        self.sandwich_k = sandwich_k

    def forward(self, src, mask = None, src_key_padding_mask = None):
        output = src
        for i, mod in enumerate(self.layers):
            if i < self.sandwich_k:
                output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, layer_type=1)
            elif i >= self.num_layers - self.sandwich_k:
                output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, layer_type=2)
            else:
                output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, layer_type=0)

        if self.norm is not None:
            output = self.norm(output)

        return output

class TransformerDecoder(nn.Module):
    _constants_ = ['norm']

    def __init__(self, decoder_layer, num_layers, norm=None, sandwich_k=0):
        super(TransformerDecoder, self).__init__()
        self.layers = _get_clones(decoder_layer, num_layers+sandwich_k)
        self.num_layers = num_layers+sandwich_k
        self.norm = norm
        self.sandwich_k = sandwich_k

    def forward(self, tgt, memory, tgt_mask = None,
                memory_mask = None, tgt_key_padding_mask = None,
                memory_key_padding_mask = None):
        output = tgt

        for i, mod in enumerate(self.layers):
            if i < self.sandwich_k:
                output = mod(output, memory, tgt_mask=tgt_mask,
                      memory_mask=memory_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask,
                      memory_key_padding_mask=memory_key_padding_mask, layer_type=1)
            elif i >= self.num_layers - self.sandwich_k:
                output = mod(output, memory, tgt_mask=tgt_mask,
                      memory_mask=memory_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask,
                      memory_key_padding_mask=memory_key_padding_mask, layer_type=2)
            else:
                output = mod(output, memory, tgt_mask=tgt_mask,
                      memory_mask=memory_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask,
                      memory_key_padding_mask=memory_key_padding_mask, layer_type=0)

        if self.norm is not None:
            output = self.norm(output)

        return output

class TransformerEncoderLayer(nn.Module):
    _constants_ = ['batch_first', 'norm_first']

    def __init__(self, d_model, nhead, dim_feedforward = 2048, dropout = 0.1,
                 activation = F.relu, layer_norm_eps = 1e-5):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = activation

    def _setstate_(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerEncoderLayer, self)._setstate_(state)

    def forward(self, src, src_mask = None, src_key_padding_mask = None,
                layer_type=0):
        x = src

        if layer_type == 0:
            x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask))
            x = self.norm2(x + self._ff_block(x))
        elif layer_type == 1:
            x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask))
        else:
            x = self.norm2(x + self._ff_block(x))

        return x

    # self-attention block
    def _sa_block(self, x, attn_mask, key_padding_mask):
        x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False)[0]
        return self.dropout1(x)

    # feed forward block
    def _ff_block(self, x):
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout2(x)

class TransformerDecoderLayer(nn.Module):
    _constants_ = ['batch_first', 'norm_first']

    def __init__(self, d_model, nhead, dim_feedforward = 2048, dropout = 0.1,
                 activation = F.relu, layer_norm_eps = 1e-5):
      
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm3 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = activation

    def _setstate_(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerDecoderLayer, self)._setstate_(state)

    def forward(self, tgt, memory, tgt_mask = None, memory_mask= None,
                tgt_key_padding_mask = None, memory_key_padding_mask = None,
                layer_type = 0):
        x = tgt
        if layer_type == 0:
            x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
            x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask))
            x = self.norm3(x + self._ff_block(x))
        elif layer_type == 1:
            x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
            x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask))
        else:
            x = self.norm3(x + self._ff_block(x))

        return x

    # self-attention block
    def _sa_block(self, x, attn_mask , key_padding_mask):
        x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False)[0]
        return self.dropout1(x)

    # multihead attention block
    def _mha_block(self, x, mem, attn_mask, key_padding_mask):
        x = self.multihead_attn(x, mem, mem,
                                attn_mask=attn_mask,
                                key_padding_mask=key_padding_mask,
                                need_weights=False)[0]
        return self.dropout2(x)

    # feed forward block
    def _ff_block(self, x):
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout3(x)

def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu

Pre-Trained Transformer Model

In [57]:
import argparse
import math
import os

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

PAD = 0
UNK = 1
EOS = 2
SOS = 3
MASK = 4

class SandwichSmTrfm(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_layers, sandwich_k, sandwich_encoder, sandwich_decoder):
        super(SandwichSmTrfm, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(in_size, hidden_size)
        self.pe = PositionalEncoding(hidden_size, 0.1)
        self.sandwich_k = sandwich_k
        self.sandwich_encoder = sandwich_encoder
        self.sandwich_decoder = sandwich_decoder
        self.trfm = SandwichTransformer(d_model=hidden_size, nhead=4, 
        num_encoder_layers=n_layers, num_decoder_layers = n_layers, dim_feedforward=hidden_size,
        sandwich_k=sandwich_k, sandwich_encoder=sandwich_encoder, sandwich_decoder=sandwich_decoder,
        dropout=0.4)
        self.out = nn.Linear(hidden_size, out_size)

    def forward(self, src):
        embedded = self.embed(src)
        embedded += self.pe(embedded)
        hidden = self.trfm(embedded, embedded)
        out = self.out(hidden)
        out = F.log_softmax(out, dim=2)
        return out

    def _encode(self, src):
        embedded = self.embed(src)
        embedded = self.pe(embedded)
        output = embedded
        if self.sandwich_encoder:
            for i in range(self.trfm.encoder.num_layers - 1):
                if i < self.sandwich_k:
                    output = self.trfm.encoder.layers[i](output, None, layer_type=1)
                elif i >= self.trfm.encoder.num_layers - self.sandwich_k:
                    output = self.trfm.encoder.layers[i](output, None, layer_type=2)
                else:
                    output = self.trfm.encoder.layers[i](output, None, layer_type=0)
            penul = output.cpu().detach().numpy()
            output = self.trfm.encoder.layers[-1](output, None, layer_type=2)
        else:
            for i in range(self.trfm.encoder.num_layers - 1):
                output = self.trfm.encoder.layers[i](output, None, layer_type=0)
            penul = output.cpu().detach().numpy()
            output = self.trfm.encoder.layers[-1](output, None, layer_type=0)
        if self.trfm.encoder.norm:
            output = self.trfm.encoder.norm(output)
        output = output.cpu().detach().numpy()
        
        return np.hstack([np.mean(output, axis=0), np.max(output, axis=0), output[0,:,:], penul[0,:,:]])
    
    def encode(self, src):
        batch_size = src.shape[1]
        if batch_size <= 100:
            return self._encode(src)
        else: # Batch is too large to load
            i = 0
            while i < batch_size:
                if o == 0:
                    out = self._encode(src[:, i:i+100])
                else:
                    out = np.concatenate([out, self._encode(src[:, i:i+100])], axis=0)
                i += 100
            return out

def evaluate(model, test_loader, vocab):
    model.eval()
    total_loss = 0
    for b, sm in enumerate(test_loader):
        sm = torch.t(sm.cuda())
        with torch.no_grad():
            output = model(sm)
        loss = F.nll_loss(output.view(-1, len(vocab)),
                               sm.contiguous().view(-1),
                               ignore_index=PAD)
        total_loss += loss.item()
    return total_loss / len(test_loader)


In [58]:
vocab = Vocab.load_vocab('/content/drive/MyDrive/data/vocab24.pkl')
df = pd.read_csv('/content/drive/MyDrive/data/chembl_24_dropped.csv')
print(len(vocab))
dataset = MyDataset(df['canonical_smiles'].values, vocab)
test_size = 10000
train, test = torch.utils.data.random_split(dataset, [len(dataset)-test_size, test_size])
train_loader = DataLoader(train, batch_size= 8 , shuffle=True, num_workers=2)
test_loader = DataLoader(test, batch_size=8, shuffle=False, num_workers=2)
print('Train size:', len(train))
print('Test size:', len(test))

45
Train size: 1712297
Test size: 10000


In [59]:
model = SandwichSmTrfm(len(vocab), 256, len(vocab), 4, 1, False, True).cuda()
n_epochs=15
criterion = nn.NLLLoss(ignore_index=PAD)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * n_epochs))

In [61]:
import pdb
from tqdm import tqdm

scaler = torch.cuda.amp.GradScaler()
best_loss = 1e10

for epoch in range(0,12):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    total_loss = 0.0
    for i, sm in enumerate(train_loader):
        sm = torch.t(sm.cuda())
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            output = model(sm)
            loss = criterion(output.view(-1, len(vocab)),sm.contiguous().view(-1))
        total_loss += float(loss)
        batch_bar.set_postfix(
                loss=f"{float(total_loss / (i + 1))}")
      
        if i > 0 and i % 5000 ==0:
            val_loss = evaluate(model, test_loader, vocab)
            print('Val {:3d}: iter {:5d} | loss {:.10f} | perplexity {:.10f}'.format(epoch, i, val_loss, math.exp(loss)))

            if val_loss < best_loss:
                pathname = '/content/drive/MyDrive/project_best.pth' 
                torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                }, pathname)
                best_loss = loss
  
            model.train()
            pathname = '/content/drive/MyDrive/project.pth' 
            torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }, pathname)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        batch_bar.update()
    batch_bar.close()
    print("Epoch {}/{}: Train Loss {:.04f}".format(
          epoch + 1,
          12,
          float(total_loss / len(train_loader))))
    torch.save({'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'scheduler_state_dict': scheduler.state_dict(),
          }, 'drive/MyDrive/project.pth')

Train:   0%|          | 292/214038 [00:14<2:26:48, 24.27it/s, loss=2.648283962910493]

KeyboardInterrupt: ignored